In [92]:
import requests
import json

from dotenv import load_dotenv
import os
from shutil import copy
from pathlib import Path
from shutil import copy
import asyncio
import io
import aiohttp
from typing import List
from PIL import Image, ImageOps
from utils import download_images
import nest_asyncio
nest_asyncio.apply()

In [224]:
load_dotenv()

username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")

In [225]:
login_url = "https://api.345.global/api/v1/login/access-token"
credentials = {"username": username, "password": password,"grant_type": "password"}
response = requests.post(login_url, data=credentials)
token_response = response.json()

token_type = token_response.get("token_type", "Bearer") 
token = token_response.get("access_token", None)

headers = {
    "Authorization": f"{token_type} {token}"
}

In [36]:
url = f"https://api.345.global/api/v1/assetJobs/paginate?skip=0&limit=10000&columns=product.uuid&assetProject=96d7be97-5f7d-4ab0-a70a-d262dbfca786"
ccbf_uuids_response = requests.get(url,headers=headers)

In [37]:
ccbf_uuids_data = ccbf_uuids_response.json()["data"]
ccbf_uuids = [item["product"]["uuid"] for item in ccbf_uuids_data]

uuids_filter = ""
from urllib.parse import quote
for uuid in ccbf_uuids:
    uuids_filter += f"uuid={uuid},"
uuids_filter = quote(uuids_filter[:-1])

In [51]:
data = []

for skip in [0,10000,20000,30000]:

    url = f"https://api.345.global/api/v1/products/paginate?skip={skip}&limit=10000&filter=organisationUuid%3Dda4508d4-771f-435d-8058-637285ea4868&includeAssets=false&withoutDeleted=true&withoutVariants=false&withoutWallpaper=true&withProvisional=false"

    response = requests.get(url, headers=headers)

    data.extend(response.json()["data"])

    print(f"Fetched {len(response.json()['data'])} products, total so far: {len(data)}")

Fetched 10000 products, total so far: 10000
Fetched 10000 products, total so far: 20000
Fetched 10000 products, total so far: 30000
Fetched 6945 products, total so far: 36945


In [46]:
url = f"https://api.345.global/api/v1/products/paginate?skip=0&limit=10000&filter=organisationUuid%3Dda4508d4-771f-435d-8058-637285ea4868%2Csource%3D345&includeAssets=false&withoutDeleted=true&withoutVariants=false&withoutWallpaper=true&withProvisional=false"

products_345_response = requests.get(url, headers=headers)

In [ ]:
all_upcs = set([product["upc"] for product in data])

In [61]:
data[0]

{'uuid': 'bc3b1c04-8715-47da-9d15-41d19116a674',
 'organisationUuid': 'da4508d4-771f-435d-8058-637285ea4868',
 'upc': '00851424008642',
 'upc11': None,
 'upc12': None,
 'upc13': None,
 'upc14': '00851424008642',
 'upcShort': None,
 'name': 'Karma Probiotic Water - Peach Lemonade | 18 FL OZ',
 'colloquialNames': None,
 'manufacturer': 'Karma Culture',
 'supplier': None,
 'brand': 'Karma',
 'category': 'Beverages',
 'description': None,
 'price': None,
 'measurement': {'width': 6.4,
  'height': 22.1,
  'depth': 6.4,
  'volume': None,
  'weight': None},
 'isBoxLike': None,
 'primaryVariantUuid': None,
 'originalProduct': None,
 'baseProduct': None,
 'modelUrl': 'models/a39223e80166003cb97bfb55cfca1ffa.glb',
 'imageFrontUrl': 'images/040d82a8db7f12c64f1d10731ceee58d.png',
 'imageLeftUrl': 'images/b891dd235d48c850eae73abd4764ed39.png',
 'imageTopUrl': 'images/588fbe504ddd0ccb500b53e4dbaa1bec.png',
 'imageBackUrl': 'images/429ea3e1b28defed4b6813cf0a8f1a26.png',
 'imageRightUrl': 'images/d38a

In [62]:
products_dict = {}

for product in data:
    if product["isPrimary"] and products_dict.get(product["uuid"],"") == "":
            products_dict[product["uuid"]] = []
    

In [70]:
upc_data = {}

for upc in all_upcs:

    if upc_data.get(upc,"")=="":
        upc_data[upc] = []
    

In [71]:
len(upc_data)

35676

In [213]:
data_345 = [p for p in data if p["source"] == "345"]
upcs_345 = [p["upc"] for p in data if p["source"] == "345"]

In [244]:
upcs_smsb

['4900000050T',
 '4900040764',
 '7297900020T',
 '4900000986T',
 '4900002385T',
 '4900000725T',
 '4900000254T',
 '4900001427T',
 '4900001426T',
 '4900001424T',
 '4900004897',
 '4900007234',
 '8616220296',
 '8616220297',
 '8616220498',
 '8616210108',
 '8616210107',
 '8616220373',
 '8616230115',
 '8616230111',
 '8616210106',
 '4900054540',
 '4900054555',
 '2500013115',
 '2500013117',
 '4900054762',
 '7297918047',
 '4900055545',
 '2500013575',
 '4900055574',
 '4900055575',
 '4900055576',
 '7199048001',
 '1003626258',
 '1003626750',
 '7800000260T',
 '1003626748',
 '9843740430',
 '9428330176',
 '3570097081',
 '7336612755T',
 '1547301036',
 '4498400242',
 '7774136032',
 '7057800019',
 '3663200891',
 '2640029000',
 '2640022760',
 '7047440006',
 '7265000133',
 '7092400100',
 '7092400162',
 '7092400165',
 '7092400079',
 '7039922600',
 '7039943220',
 '7039964095',
 '7911714242',
 '7911710707',
 '7911710505',
 '7911719404',
 '7911717070',
 '7911720808',
 '7911720909',
 '2544430003',
 '7911712525',

In [241]:
upcs_smsb = [p["customId"] for p in data if p["source"] != "345"]

In [227]:
upcs_mismatch = set(upcs_345) - set(upcs_smsb)



In [228]:
upcs_filter = ""
from urllib.parse import quote
for upc in upcs_mismatch:
    upcs_filter += f"upc={upc},"
upcs_filter = quote(upcs_filter[:-1])

url = f"https://api.345.global/api/v1/products/paginate?skip=0&limit=10000&filter=organisationUuid%3Dda4508d4-771f-435d-8058-637285ea4868%2C{upcs_filter}&includeAssets=false&withoutDeleted=true&withoutVariants=false&withoutWallpaper=true&withProvisional=false"

response = requests.get(url, headers=headers)


In [229]:
upc_counts = {}
custom_id_counts = {}
for product in response.json()["data"]:
    upc = product["upc"]
    custom_id = product["customId"]
    
    if upc not in upc_counts:
        upc_counts[upc] = 0
    if custom_id not in custom_id_counts:
        custom_id_counts[custom_id] = 0
    
    upc_counts[upc] += 1
    custom_id_counts[custom_id] += 1

In [237]:
{k:v for k, v in sorted(upc_counts.items(),key=lambda x: x[1],reverse=True) if v > 1}

{'00049000028201': 5,
 '00012000171635': 5,
 '00049000040869': 4,
 '00818094005777': 4,
 '00810085816898': 4,
 '00858176002317': 3,
 '00049000053418': 3,
 '00049000026566': 3,
 '00070847898245': 3,
 '00049000019162': 3,
 '00049000031171': 3,
 '00049000077858': 3,
 '00049000026207': 3,
 '00815154026772': 3,
 '00012000001802': 3,
 '00611269991000': 3,
 '00818094005807': 3,
 '00818094005746': 3,
 '00818094005791': 3,
 '00012000002946': 3,
 '00012000007590': 3,
 '00049000555189': 3,
 '00049000555417': 3,
 '7343000503': 3,
 '00012000161155': 3,
 '00016571910310': 3,
 '00818094005906': 3,
 '00078742226590': 3,
 '00049000026276': 3,
 '00070847898146': 3,
 '00860004385400': 3,
 '00610764863430': 3,
 '00888903201027': 2,
 '00860009872486': 2,
 '00051202008868': 2,
 '00193968072544': 2,
 '00198168440636': 2,
 '00070847000037': 2,
 '00070847811169': 2,
 '00070847012474': 2,
 '00072979004174': 2,
 '00858176002300': 2,
 '00858176002324': 2,
 '00049000023923': 2,
 '00850009942432': 2,
 '000490000778

In [106]:
data_smsb = [p for p in data if p["upc"] in upcs_345 and p["source"] != "345"]

In [226]:
upcs_filter = ""
for upc in upcs_smsb:
    upcs_filter += f"upc={upc},"

upcs_filter = quote(upcs_filter[:-1])

In [90]:
url = f"https://api.345.global/api/v1/products/paginate?skip=0&limit=10000&filter=organisationUuid%3Dda4508d4-771f-435d-8058-637285ea4868%2C{upcs_filter}&includeAssets=false&withoutDeleted=true&withoutVariants=false&withoutWallpaper=true&withProvisional=false"

response = requests.get(url, headers=headers)

In [242]:
combined_upcs = [p for p in data if p["upc"] in upcs_345 and p["customId"] in upcs_smsb]

In [243]:
len(combined_upcs)

1320

In [207]:
combined_upcs

[{'uuid': 'c1337875-442e-43e4-9d7a-8b35373a1615',
  'organisationUuid': 'da4508d4-771f-435d-8058-637285ea4868',
  'upc': '5200004774',
  'upc11': None,
  'upc12': None,
  'upc13': None,
  'upc14': None,
  'upcShort': None,
  'name': 'Gatorade Zero Fruit Punch 20oz',
  'colloquialNames': None,
  'manufacturer': 'Pepsi',
  'supplier': None,
  'brand': 'Gatorade Zero',
  'category': 'Soft Beverages',
  'description': None,
  'price': None,
  'measurement': {'width': 7.493,
   'height': 19.6088,
   'depth': 7.493,
   'volume': None,
   'weight': None},
  'isBoxLike': None,
  'primaryVariantUuid': None,
  'originalProduct': None,
  'baseProduct': None,
  'modelUrl': None,
  'imageFrontUrl': None,
  'imageLeftUrl': None,
  'imageTopUrl': None,
  'imageBackUrl': None,
  'imageRightUrl': None,
  'imageBottomUrl': None,
  'customId': '5200004774',
  'vendor': None,
  'department': None,
  'subCategory': 'NCB',
  'status': None,
  'part': None,
  'replenished': False,
  'disposition': None,
  'a

In [200]:
data[0]["customId"]

'5142400864'

In [123]:
len([p for p in response.json()["data"] if p["uuid"] == "4fab52af-4075-44ec-ad51-6195792e6ccd"])

1

In [134]:
product_counts = {}
for product in combined_upcs:
    if product["upc"] not in product_counts:
        product_counts[product["upc"]] = 0
    product_counts[product["upc"]] += 1

In [ ]:
faceint_to_str = {1: "front", 2: "left", 7: "back", 8: "right"}

os.makedirs("planogramImages/smsb", exist_ok=True)
os.makedirs("planogramImages/345", exist_ok=True)
from tqdm import tqdm
for product in tqdm(combined_upcs):
    image_urls = []
    image_sides = []
    if product_counts[product["upc"]] == 1 or product["source"] == "345":
        product["isPrimary"] = False

    if product["isPrimary"]:
        for image in product["planogramImages"]:
            face = faceint_to_str.get(image["face"],"")
            if face == "":
                continue
            if 
        #     image_urls.append(f"{image["url"]}/{image["folder"]}/{image["fileHashMd5"]}.{image["extension"]}")
        #     image_sides.append(face)
        # images = asyncio.run(download_images(image_urls))
        # for image, side in zip(images, image_sides):
        #     image.save(f"planogramImages/smsb/{product['uuid']}_{product['upc']}_{side}.jpg")


100%|██████████| 1420/1420 [05:54<00:00,  4.01it/s] 


In [137]:
import os
import json

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import torch

from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from collections import defaultdict
from tqdm import tqdm
from embedder import Embedder
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from torchvision import transforms

/Users/iman/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [142]:
ROOT_DIR = "/Users/iman/345-data/"
DATA_DIR = Path(ROOT_DIR, "ml-datasets", "smsb")
VECTORS_DIR = Path(DATA_DIR, "vectors")

DATASET = "smsb-20250708"
output_dir = VECTORS_DIR / DATASET
crops_dir = DATA_DIR / "recognition" / DATASET
output_dir.mkdir(parents=True, exist_ok=True)
crops_dir.mkdir(parents=True, exist_ok=True)

In [139]:
device = torch.device("mps")

In [140]:
model_root = Path("./model")


embedder = Embedder(model_root / "trunk_weights.pth",
                    model_root / "embedder_weights.pth",device=device)

In [143]:

to_tensor = transforms.ToTensor()
resize = transforms.Resize((224, 224),interpolation=transforms.InterpolationMode.BILINEAR)
images = [Image.open(img) for img in sorted((crops_dir).rglob("*.jpg"))]
images_tensors = torch.stack([resize(to_tensor(img)) for img in images]).to(device)


In [144]:
images_tensors.shape

torch.Size([2341, 3, 224, 224])

In [145]:
vectors = embedder.embed(images_tensors)

In [ ]:
torch.save(vectors, output_dir / "supervectors.pt")

In [161]:
image_names = [img.stem for img in sorted((crops_dir).rglob("*.jpg"))]
classes = ["_".join(name.split("_")) for name in image_names]
json.dump(image_names, open(output_dir / "classes.json", "w"))

In [164]:
ref_vectors = vectors
ref_classes = image_names

In [194]:
import json

with open ("/Users/iman/Downloads/assets/345-asset-library-upcs-uuids.json") as f:
    upcs_uuids = json.load(f)["data"]



In [195]:
import pandas as pd
upcs_uuids_df = pd.DataFrame(upcs_uuids)
upcs_uuids_df.to_csv(Path("/Users/iman/Downloads/assets") / "upcs_uuids.csv", index=False)

In [152]:
ref_vectors.shape

torch.Size([2341, 256])

In [ ]:


## need to normalize the supervectors
unique_classes, class_indices = np.unique(np.array(ref_classes), return_inverse=True)
unique_uuids, uuid_indices = np.unique(
    ["_".join(cls.split("_")[:-1]) for cls in ref_classes], return_inverse=True
)
num_of_classes = class_indices[-1] + 1
num_uuids = uuid_indices[-1] + 1
#supervectors = torch.empty((num_of_classes, 256), dtype=ref_vectors.dtype)
supervectors = torch.empty((num_of_classes, 256))
for class_i in tqdm(range(num_of_classes)):
    vectors = ref_vectors[class_indices == class_i]
    supervectors[class_i, :] = torch.mean(vectors, axis=0)

supervectors /= torch.linalg.norm(supervectors, axis=1)[:, torch.newaxis]
superclasses = unique_classes

100%|██████████| 1058/1058 [00:01<00:00, 539.42it/s]


In [162]:
torch.save(
    supervectors,
    output_dir / "supervectors.pt",
)
with open(
    output_dir / "superclasses.json", "w"
) as f:
    json.dump(superclasses.tolist(), f)

NameError: name 'supervectors' is not defined

In [165]:
def train_sample_split(vectors, classes, face=None):
    print(vectors.shape, classes.shape)
    if isinstance(vectors,torch.Tensor):
        vectors = np.array(vectors.cpu().numpy())
        classes = np.array(classes)
    # if face is not None, create a mask for the face, applied to all
    if face is not None:
        vectors, classes = filter_by_face(vectors, classes, face)
    print(vectors.shape, classes.shape)
    # ## offset elements by 1 so that the last element of each face is not included in training set
    mask = np.concatenate(([False], classes[:-1] != classes[1:]))
    # split the vectors and classes into train and sample sets
    print(vectors.shape, classes.shape, mask.shape)
    train_vectors = vectors[~mask, :]
    train_classes = classes[~mask]

    sample_vectors = vectors[mask, :]
    sample_classes = classes[mask]

    return train_vectors, train_classes, sample_vectors, sample_classes

def create_face_mask(classes, face):
    if isinstance(face, (tuple, list)):
        return np.array([any(f in label for f in face) for label in classes])
    else:
        return np.array([face in label for label in classes])

def filter_by_face(vectors, classes, face):
    face_mask = np.array([cls.split("_")[-1] == face for cls in classes])
    return vectors[face_mask], np.array(classes)[face_mask]



In [184]:
## train knn for specific faces
sample_vectors = torch.load('/Users/iman/345-data/ml-datasets/ccbf/vectors/ccbf-20241127-20250326/test set/testvectors.pt')
sample_classes = json.load(open('/Users/iman/345-data/ml-datasets/ccbf/vectors/ccbf-20241127-20250326/test set/classes.json', 'r'))

sample_vectors = sample_vectors.cpu().numpy()
sample_classes = np.array(sample_classes)

ref_vectors = vectors
ref_classes = image_names

for face in ("front", "left", "right", "back", None):
    if face:
        train_vectors, train_classes = filter_by_face(ref_vectors.cpu().numpy(), ref_classes, face)
    else:
        train_vectors = ref_vectors.cpu().numpy()
        train_classes = np.array(ref_classes)
    # fit knn to ref set
    knn = KNeighborsClassifier(n_neighbors=1, metric="euclidean")
    knn.fit(train_vectors, train_classes)

    predicted_labels = knn.predict(sample_vectors)
    print([(gt, pred) for gt, pred in zip(sample_classes[:3], predicted_labels[:3])])
    train_upcs = [cls.split("_")[1] for cls in train_classes]
    correct_predictions = sum(
        [
            1
            for gt, pred in zip(sample_classes, predicted_labels)
            if gt.split("_")[1] == pred.split("_")[1] if gt.split("_")[1] in train_upcs
        ]
    )
    total_test_classes = [cls.split("_")[1] for cls in sample_classes if cls.split("_")[1] in train_upcs]
    print(len(total_test_classes))
    accuracy = correct_predictions / len(total_test_classes)

    print(f"Accuracy for {face=}: {accuracy * 100:.2f}%")


# which ones fail?
len(sample_classes) - correct_predictions

incorrect_predictions = [
    (gt, pred)
    for gt, pred in zip(sample_classes, predicted_labels)
    if gt.split("_")[1] != pred.split("_")[1]
]

[('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_front', 'bf906b27-9d58-4a9b-8602-e395ebf49bef_00021136180596_front'), ('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_left', 'bf906b27-9d58-4a9b-8602-e395ebf49bef_00021136180596_front'), ('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_right', 'bf906b27-9d58-4a9b-8602-e395ebf49bef_00021136180596_front')]
1308
Accuracy for face='front': 77.60%
[('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_front', 'bf906b27-9d58-4a9b-8602-e395ebf49bef_00021136180596_left'), ('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_left', 'bf906b27-9d58-4a9b-8602-e395ebf49bef_00021136180596_left'), ('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_right', 'bf906b27-9d58-4a9b-8602-e395ebf49bef_00021136180596_left')]
1254
Accuracy for face='left': 61.24%
[('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_front', '91c5e7e6-7b6b-4a54-b797-3f47bd512519_00074806001615_right'), ('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_lef

In [185]:
## train knn for specific faces
train_vectors = torch.load('/Users/iman/345-data/ml-datasets/ccbf/vectors/ccbf-20241127-20250326/reference/refvectors.pt')
train_classes = json.load(open('/Users/iman/345-data/ml-datasets/ccbf/vectors/ccbf-20241127-20250326/reference/classes.json', 'r'))

ref_vectors = train_vectors
ref_classes = train_classes

for face in ("front", "left", "right", "back", None):
    if face:
        train_vectors, train_classes = filter_by_face(ref_vectors.cpu().numpy(), ref_classes, face)
    else:
        train_vectors = ref_vectors.cpu().numpy()
        train_classes = np.array(ref_classes)
    # fit knn to ref set
    knn = KNeighborsClassifier(n_neighbors=1, metric="euclidean")
    knn.fit(train_vectors, train_classes)

    predicted_labels = knn.predict(sample_vectors)
    print([(gt, pred) for gt, pred in zip(sample_classes[:3], predicted_labels[:3])])
    train_upcs = [cls.split("_")[1] for cls in train_classes]
    correct_predictions = sum(
        [
            1
            for gt, pred in zip(sample_classes, predicted_labels)
            if gt.split("_")[1] == pred.split("_")[1] if gt.split("_")[1] in train_upcs
        ]
    )
    
    total_test_classes = [cls.split("_")[1] for cls in sample_classes if cls.split("_")[1] in train_upcs]
    print(len(total_test_classes))
    accuracy = correct_predictions / len(total_test_classes)

    print(f"Accuracy for {face=}: {accuracy * 100:.2f}%")


# which ones fail?
len(sample_classes) - correct_predictions

incorrect_predictions = [
    (gt, pred)
    for gt, pred in zip(sample_classes, predicted_labels)
    if gt.split("_")[1] != pred.split("_")[1]
]

[('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_front', 'c2c64acb-2391-4620-8e98-6b895f1318f8_00021136010374_front'), ('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_left', '007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_front'), ('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_right', '007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_front')]
2565
Accuracy for face='front': 87.21%
[('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_front', 'c2c64acb-2391-4620-8e98-6b895f1318f8_00021136010374_left'), ('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_left', '007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_left'), ('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_right', 'c2c64acb-2391-4620-8e98-6b895f1318f8_00021136010374_left')]
2565
Accuracy for face='left': 76.65%
[('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_front', 'c2c64acb-2391-4620-8e98-6b895f1318f8_00021136010374_right'), ('007a6be0-4ed0-4950-88e7-dd0547f50127_00021136180596_lef

In [188]:
Path("Users/iman") / ""

PosixPath('Users/iman')

In [189]:
path = ""
if path:
    print("yes")